In [23]:
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob
from tensorflow.keras.utils import img_to_array 
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input

In [24]:
ROOT_DIR = "C:/Users/jpsha/Documents/capstone/colonDataSet"
number_of_images = {}
for dir in os.listdir(ROOT_DIR):
    number_of_images[dir] = len(os.listdir(os.path.join(ROOT_DIR,dir)))
number_of_images.items()

dict_items([('no', 15), ('yes', 15)])

In [25]:
os.listdir("C:/Users/jpsha/Documents/capstone/colonDataSet")

['no', 'yes']

In [26]:
def dataFolder(p,split):
    if not os.path.exists("./"+p):
        os.mkdir("./"+p)
        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./"+p+"/"+dir)
            for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR,dir)),
                                       size = (math.floor(split*number_of_images[dir])-5),
                                        replace = False
                                       ):
                O = os.path.join(ROOT_DIR, dir, img)
                D = os.path.join("./"+p,dir)
                shutil.copy(O,D)
                os.remove(O)
    else:
        print(f"{p}Folder exist")

In [27]:
dataFolder("train",0.7)

trainFolder exist


In [28]:
dataFolder("val",0.15)

valFolder exist


In [29]:
dataFolder("test",0.15)

testFolder exist


In [30]:
def preprocessingImages1(path):
    image_data = ImageDataGenerator(zoom_range=0.2, shear_range=0.2, preprocessing_function= preprocess_input, horizontal_flip=True)
    image = image_data.flow_from_directory(path, target_size=(224,224),batch_size=32,class_mode = 'binary')
    return image

In [31]:
path = "C:/Users/jpsha/Documents/capstone/train"
train_data = preprocessingImages1(path)

Found 6990 images belonging to 2 classes.


In [32]:
def preprocessingImages2(path):
    image_data = ImageDataGenerator(preprocessing_function= preprocess_input) 
    image = image_data.flow_from_directory(path, target_size=(224,224),batch_size=32, class_mode='binary')
    return image

In [33]:
path = "C:/Users/jpsha/Documents/capstone/test"
test_data = preprocessingImages2(path)

Found 1490 images belonging to 2 classes.


In [34]:
path = "C:/Users/jpsha/Documents/capstone/val"
val_data = preprocessingImages2(path)

Found 1490 images belonging to 2 classes.


In [35]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense
from keras.models import Model, load_model


In [36]:
base_model = MobileNet(input_shape=(224,224,3), include_top=False)

17225924/17225924 [==============================] - 2s 0us/step


In [37]:
for layer in base_model.layers:
    layer.trainable = False

In [38]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [39]:
X = Flatten()(base_model.output)
X = Dense(units=1, activation='sigmoid')(X)

model = Model(base_model.input, X)

In [40]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [41]:
import keras

In [42]:
model.compile(optimizer='rmsprop', loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [43]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

mc = ModelCheckpoint(filepath="C:/Users/jpsha/Documents/capstone/bestmodel.h5", monitor='val_accuracy',verbose=1, save_best_only=True)

es = EarlyStopping(monitor='val_accuracy',min_delta=0.01, patience=3, verbose=1)

cb = [mc, es]

In [44]:
hist = model.fit_generator(train_data,
                          steps_per_epoch=8,
                           epochs=30,
                           validation_data=val_data,
                           validation_steps=16,
                           callbacks = cb
                          )

C:\Users\jpsha\AppData\Local\Temp\ipykernel_28116\3084188717.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(train_data,


Epoch 1/30
8/8 [==============================] - ETA: 0s - loss: 5.4059 - accuracy: 0.7227
Epoch 1: val_accuracy improved from -inf to 0.97461, saving model to C:/Users/jpsha/Documents/capstone\bestmodel.h5
8/8 [==============================] - 16s 2s/step - loss: 5.4059 - accuracy: 0.7227 - val_loss: 0.0949 - val_accuracy: 0.9746
Epoch 2/30
8/8 [==============================] - ETA: 0s - loss: 0.0629 - accuracy: 0.9805
Epoch 2: val_accuracy improved from 0.97461 to 0.98438, saving model to C:/Users/jpsha/Documents/capstone\bestmodel.h5
8/8 [==============================] - 15s 2s/step - loss: 0.0629 - accuracy: 0.9805 - val_loss: 0.0473 - val_accuracy: 0.9844
Epoch 3/30
8/8 [==============================] - ETA: 0s - loss: 0.2008 - accuracy: 0.9609
Epoch 3: val_accuracy improved from 0.98438 to 0.99609, saving model to C:/Users/jpsha/Documents/capstone\bestmodel.h5
8/8 [==============================] - 16s 2s/step - loss: 0.2008 - accuracy: 0.9609 - val_loss: 0.0115 - val_accura

In [45]:
model = load_model("C:/Users/jpsha/Documents/capstone/bestmodel.h5")

In [46]:
acc = model.evaluate_generator(test_data)[1]
print(f"our mode accuracy is {acc * 100}%")

C:\Users\jpsha\AppData\Local\Temp\ipykernel_28116\1580348637.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc = model.evaluate_generator(test_data)[1]


our mode accuracy is 98.65771532058716%
